# Worksheet 08

Name:  
UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [40]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.637922007726626, 5.584564731905413, 5.2785517973546465, 4.425963711200415, 5.061161448530095, 5.59325556502887, 5.194120355754815, 5.5533498484366755, 4.685676105796061, 3.6278612568744952]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [41]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[6.807890528475329, 7.131392774068784, 9.67836238879045, 6.343646919405394, 8.256137837318795, 7.242229755335708, 8.075698485396202, 7.892125034391276, 9.09558688350922, 6.742296826428417]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [42]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[6.742296826428417, 9.09558688350922, 7.892125034391276, 8.075698485396202, 3.6278612568744952, 7.242229755335708, 8.256137837318795, 4.685676105796061, 5.5533498484366755, 6.343646919405394]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

We know the mean and variance of both datasets, that is $\mu_1 = 5, \mu_2 = 8, \sigma_1 = \sigma_2 = 1$. We also know that N = 10 for both datasets

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [43]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) ,  len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[6.742296826428417, 9.09558688350922, 7.892125034391276, 8.075698485396202, 7.242229755335708, 8.256137837318795, 6.343646919405394]
[3.6278612568744952, 4.685676105796061, 5.5533498484366755]
P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 7.66396024882643,  mean_2 = 4.622295737035743
var_1 = 0.7703377176349212,  var_2 = 0.6199262549447816


/home/timevdo/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


They are generally, very wrong. The standard distributions are off by anywhere from 25-50%, the means are actually kinda close, and the probabilities are also pretty wrong

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [44]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  6.742296826428417
probability of observing that point if it came from cluster 0 =  0.2531564810544354
probability of observing that point if it came from cluster 1 =  0.0018581489345639596
point =  9.09558688350922
probability of observing that point if it came from cluster 0 =  0.09209704851477758
probability of observing that point if it came from cluster 1 =  3.1773103166020893e-12
point =  7.892125034391276
probability of observing that point if it came from cluster 0 =  0.49565461674663436
probability of observing that point if it came from cluster 1 =  5.852659958082432e-07
point =  8.075698485396202
probability of observing that point if it came from cluster 0 =  0.44894614374127023
probability of observing that point if it came from cluster 1 =  1.1748568373913461e-07
point =  3.6278612568744952
probability of observing that point if it came from cluster 0 =  5.666007059546412e-07
probability of observing that point if it came from cluster 1 =  0.1777494922755899
point

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [45]:
prob_s = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.7070841402937497,  P(S_2) = 0.29291585970625034
mean_1 = 7.636915049599311,  mean_2 = 4.614019139497136
var_1 = 0.7710691604361559,  var_2 = 0.6199947570115977


Probabilities are okay-ish, means are very close, variances much smaller

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [48]:
for _ in range(100):
    prob_s0_x = [] # P(S_0 | X_i)
    prob_s1_x = [] # P(S_1 | X_i)
    prob_x = [] # P(X_i)

    k = 2

    for p in data:
        pdf_i = []

        for j in range(k):
            # P(X_i | S_j)
            pdf_i.append(norm.pdf(p, mean[j], var[j]))
            # P(S_j) already computed
            prob_s[j]

        # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
        prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

        # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
        prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
        prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )
    
    prob_s = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
    mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
    var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.7111275827252277,  P(S_2) = 0.2888724172747722
mean_1 = 7.62621608650294,  mean_2 = 4.598044690056813
var_1 = 0.7717623394244233,  var_2 = 0.6205143682243559


They converge!

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [50]:
for i, point in enumerate(data):
    if prob_s0_x[i] > 0.5:
        print("Point ", point, " probably in S0 with probability ", prob_s0_x[i])
    else:
        print("Point ", point, " probably in S1", prob_s1_x[i])

Point  6.742296826428417  probably in S0 with probability  0.9975210890599466
Point  9.09558688350922  probably in S0 with probability  0.9999999999878983
Point  7.892125034391276  probably in S0 with probability  0.9999995928774854
Point  8.075698485396202  probably in S0 with probability  0.9999999095235901
Point  3.6278612568744952  probably in S1 0.9999900245469541
Point  7.242229755335708  probably in S0 with probability  0.9999348148889715
Point  8.256137837318795  probably in S0 with probability  0.9999999799829079
Point  4.685676105796061  probably in S1 0.9985945039523811
Point  5.5533498484366755  probably in S1 0.8505812363177888
Point  6.343646919405394  probably in S0 with probability  0.9629862057486024
